Importando bibliotecas

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA

Importando Base de Dados

In [ ]:
df = pd.read_csv("../databases/ENEM_2023_FINAL_cat.csv")
display(df.sample(5))

Definindo funções que realizam testes de anova e manova

In [ ]:
def anova_test(df:pd.DataFrame, y_col:str, x_cols:list[str]):
    ols_formula = f"{y_col} ~ {' * '.join(f'C({item})' for item in x_cols)}"
    print(ols_formula)
    ols_model = ols(formula=ols_formula, data=df).fit()
    display(sm.stats.anova_lm(ols_model, typ=2))

def anova_test_for_all_grades(df:pd.DataFrame, x_cols:list[str]):
    grade_cols = ["NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_MT", "NU_NOTA_LC", "NU_NOTA_REDACAO"]
    for col in grade_cols:
        anova_test(df, col, x_cols)

def manova_test(df: pd.DataFrame, y_cols: list[str], x_cols: list[str]):
    manova_formula = f"{' + '.join(y_cols)} ~ {' + '.join(x_cols)}"
    manova_model = MANOVA.from_formula(manova_formula, data=df)
    mv_results = manova_model.mv_test()
    
    dfs = {}
    for factor, result in mv_results.results.items():
        stat_df = pd.DataFrame(result['stat'])
        stat_df.index.name = factor
        dfs[factor] = stat_df.reset_index()
    
    for factor, df_factor in dfs.items():
        display(df_factor)

def manova_test_for_all_grades(df:pd.DataFrame, x_cols:list[str]):
    grade_cols = ["NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_MT", "NU_NOTA_LC", "NU_NOTA_REDACAO"]
    manova_test(df, grade_cols, x_cols)
    

ANOVA - Sexo por Notas

In [ ]:
anova_test_for_all_grades(df, ["TP_SEXO"])

ANOVA - Cor/raça por nota

In [ ]:
anova_test_for_all_grades(df, ["TP_COR_RACA"])

ANOVA - Escolaridade mãe e escolaridade pai por Notas

In [ ]:
anova_test_for_all_grades(df, ["ESCOLARIDADE_MAE", "ESCOLARIDADE_PAI"])

ANOVA - Dependencia da Escola por Nota

In [ ]:
anova_test_for_all_grades(df, ["TP_DEPENDENCIA_ADM_ESC"])

ANOVA - Renda mensal casa por Notas

In [ ]:
anova_test_for_all_grades(df, ["RENDA_MENSAL_CASA"])

ANOVA - Internet por Notas

In [ ]:
anova_test_for_all_grades(df, ["INTERNET_CASA"])

MANOVA - Sexo e cor/raça para notas

In [ ]:
manova_test_for_all_grades(df, ["TP_SEXO", "TP_COR_RACA"])

MANOVA - Notas por características familiares

In [ ]:
manova_test_for_all_grades(df, ["ESCOLARIDADE_MAE", "ESCOLARIDADE_PAI"])

MANOVA - Notas e dispositivos na casa

In [ ]:
manova_test_for_all_grades(df, ["N_CELULAR_CASA", "N_COMP_CASA", "INTERNET_CASA"])